# Sparkify Analytics 

<b>Sparkify</b> is an international media services provider. It's primary business is to build an <b>audio streaming</b> platform that provides music, videos and podcasts from record labels and media companies  

## Challenge

Sparkify wants to better serve its users and thus needs to analyze the data collected on songs and user activity on their music streaming app. The analytical goal is to understand what songs users are listening to  

## Architecture 

Currently, the data resides in a directory of CSV logs on user activity on the app  
This architecture doesn't provide an easy way to query the data  

## Analytics goals 

Sparkify wants to create a <b>Cassandra database</b> with tables designed to optimize queries on song play analysis. The main idea is to create a <b>(OLAP-oriented) database schema</b> to support their analytical needs and <b>ETL pipeline</b> to populate it from their logs  
Below is a 2-step process, where first all the CSV files are aggregated into a single CSV file and second the tables are created and populated according to the queries the data is modeled after

## Part I. ETL Pipeline for pre-processing the files

<b>The following code must be run in order to pre-process the files</b>

#### Import Python packages 

In [21]:
# Import Python packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Create constants for the notebook

In [22]:
# define constants
EVENT_FOLDER = '/event_data'
EVENT_FILE = 'event_datafile_new.csv'

#### Create a list of filepaths to process original event csv data files

In [23]:
# checking your current working directory
print(os.getcwd())

# get your current folder and subfolder event data
filepath = os.getcwd() + EVENT_FOLDER

# create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Process the files to create the data file csv that will be used for Apache Cassandra tables

In [24]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []

# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:
        # creating a csv reader object
        csvreader = csv.reader(csvfile)
        next(csvreader)

        # extracting each data row one by one and append it
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# create a new event data csv file called event_datafile_new that will be used to insert data into the Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open(EVENT_FILE, 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])

    for row in full_data_rows_list:
        # if the row does not include an artist then discard it
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [25]:
# check the number of rows in your csv file
with open(EVENT_FILE, 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra Analytical Process

<b>The CSV file titled <font color=red>event_datafile_new.csv</font> located within the Workspace directory is now available</b>  

It contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## 1. Run the following code to perform the initial setup in order to use Apache Cassandra

#### Create a Cluster

In [26]:
# create a connection to a Cassandra instance on a local server (127.0.0.1)
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    # create a session to establish a connection and begin executing queries
    session = cluster.connect()
    # report success
    print("Successfully connected to the cluster")
except Exception as e:
    # report error if any
    print(e)

Successfully connected to the cluster


#### Create a Keyspace

In [27]:
# create a keyspace
try:
    session.execute(" \
        CREATE KEYSPACE IF NOT EXISTS udacity \
        WITH REPLICATION = \
            { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }")
    # report success
    print("Keyspace successfully created")
except Exception as e:
    # report error if any
    print(e)

Keyspace successfully created


#### Set the Keyspace

In [28]:
# set current keyspace to the keyspace created above
try:
    session.set_keyspace('udacity')
    # report success
    print("Keyspace set successfully")
except Exception as e:
    # report error if any
    print(e)

Keyspace set successfully


<b>Now we are ready to create tables to run queries</b>

## The queries must answer the following 3 questions of the data

<b>1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4</b>


<b>2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182</b>
    

<b>3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'</b>




## 2. How should we model this data to run query 1 ?

`Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4`

Since your data is looking for the `sessionId` and `itemInSession`, let's start with that. Their combination uniquely identifies a song that was heard in the dataset. They are good candidates for the <b>primary key</b>  

The `sessionId` by itself is enough to spread the data evenly across nodes so it will be the <b>partition key</b>, `itemInSession` will be a <b>clustering column</b>

`artist`, `song title` and `song's length` are additional columns in the table


### Create the table

In [29]:
# drop the table if already exists
query = "DROP TABLE IF EXISTS sessions_history"

# create the songs_by_session table
try:
    session.execute(query)
    # report success
    print("Table songs_by_session successfully deleted")
except Exception as e:
    # report error if any
    print(e)

query = "CREATE TABLE IF NOT EXISTS songs_by_session "
query = query + "(session_id int, item_in_session int, artist text, title text, length float, "
query = query + "PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
    # report success
    print("Table songs_by_session successfully (re)created")
except Exception as e:
    # report error if any
    print(e)


Table songs_by_session successfully deleted
Table songs_by_session successfully (re)created


### Insert data into the table

In [30]:
with open(EVENT_FILE, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert data into table
        query = "INSERT INTO songs_by_session (session_id, item_in_session, artist, title, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

print("Data inserted")

Data inserted


### Validate the model with the query  
`SELECT artist, title, length FROM songs_by_session WHERE session_id=338 AND item_in_session=4`

In [31]:
# run the query against the model
query = "SELECT artist, title, length \
    FROM songs_by_session \
    WHERE session_id=338 AND item_in_session=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.title, row.length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


<b>Success, It worked !!</b>

## 3. How should we model this data to run query 2 ?

`Give me the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182`

Since your data is looking for the `userId` and `sessionId`, let's start with that. Unfortunately their combination does not uniquely identify a song that was heard in the dataset. Moreover, the `artist` and `song` must be sorted by `itemInSession`.  
So a combination of these 3 fields is a good candidate for the <b>primary key</b>  

The `userId` by itself is enough to spread the data evenly across nodes so it will be the <b>partition key</b>, `sessionId` and `itemInSession` will be <b>clustering columns</b> 

`artist`, `song title`, `first name` and `last name` are additional columns in the table

### Create the table

In [12]:
# drop the table if already exists
query = "DROP TABLE IF EXISTS songs_by_user_session"

try:
    session.execute(query)
    # report success
    print("Table songs_by_user_session successfully deleted")
except Exception as e:
    # report error if any
    print(e)

# create the songs_by_user_session table
query = "CREATE TABLE IF NOT EXISTS songs_by_user_session "
query = query + "(user_id int, session_id int, item_in_session int, artist text, title text, first_name text, last_name text, "
query = query + "PRIMARY KEY ((user_id, session_id), item_in_session))"

try:
    session.execute(query)
    # report success
    print("Table songs_by_user_session successfully (re)created")
except Exception as e:
    # report error if any
    print(e)


Table songs_by_user_session successfully deleted
Table songs_by_user_session successfully (re)created


### Insert data into the table

In [13]:
with open(EVENT_FILE, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert data into table
        query = "INSERT INTO songs_by_user_session (user_id, session_id, item_in_session, artist, title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

print("Data inserted")

### Validate the model with the query  
`SELECT SELECT artist, title, first_name, last_name FROM songs_by_user_session WHERE user_id = 10 AND session_id = 182 ORDER BY session_id, item_in_session ASC`

In [15]:
# run the query against the model
query = "SELECT artist, title, first_name, last_name \
    FROM songs_by_user_session \
    WHERE user_id = 10 AND session_id = 182 \
    ORDER BY item_in_session ASC"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


<b>Success, It worked !!</b>

## 4. How should we model this data to run query 3 ?

`Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'`

Since your data is looking for the `title`, let's start with that. Unfortunately the title of a song by itself does not uniquely identify a user in the dataset.  
So a combination of these 2 fields is a good candidate for the <b>primary key</b>  

The `title` by itself is enough to spread the data evenly across nodes so it will be the <b>partition key</b>, `userId` will be a <b>clustering column</b> 

`first name` and `last name` are additional columns in the table

### Create the table

In [16]:
# drop the table if already exists
query = "DROP TABLE IF EXISTS songs_by_user"
try:
    session.execute(query)
    # report success
    print("Table songs_by_user successfully deleted")
except Exception as e:
    # report error if any
    print(e)

# create the songs_by_user table
query = "CREATE TABLE IF NOT EXISTS songs_by_user "
query = query + "(title text, user_id int, first_name text, last_name text, "
query = query + "PRIMARY KEY (title, user_id))"

try:
    session.execute(query)
    # report success
    print("Table songs_by_user successfully (re)created")
except Exception as e:
    # report error if any
    print(e)


Table songs_by_user successfully deleted
Table songs_by_user successfully (re)created


### Insert data into the table

In [17]:
with open(EVENT_FILE, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert data into table
        query = "INSERT INTO songs_by_user (title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"

        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

print("Data inserted")

### Validate the model with the query  
`SELECT first_name, last_name FROM songs_by_user WHERE title = 'All Hands Against His Own'`

In [18]:
# run the query against the model
query = "SELECT first_name, last_name \
    FROM songs_by_user \
    WHERE title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


<b>Success, It worked !!</b>

### Drop the tables before closing out the session

In [19]:
query1 = "DROP TABLE IF EXISTS songs_by_session"
query2 = "DROP TABLE IF EXISTS songs_by_user_session"
query3 = "DROP TABLE IF EXISTS songs_by_user"

try:
    session.execute(query1)
    # report success
    print("Table songs_by_session successfully deleted")
    session.execute(query2)
    # report success
    print("Table songs_by_user_session successfully deleted")
    session.execute(query3)
    # report success
    print("Table songs_by_user successfully deleted")
except Exception as e:
    # report error if any
    print(e)


Table songs_by_session successfully deleted
Table songs_by_user_session successfully deleted
Table songs_by_user successfully deleted


### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()